## Thinking1：在推荐系统中，FM和MF哪个应用的更多，为什么

 - FM 应用的更加多一点，因为FM 相对于 MF 来说考虑的维度更加的多，更加贴合实际工业界的应用场景

## Thinking2：FFM与FM有哪些区别？

FFM 更加精确的计算了 Vi 和 Vj,具体来说：
1. FM算法
- 每个特征只有一个隐向量
- FM 是 FFM 的特列
- 计算速度较快，准确性较差

2. FFM 算法
- 每个特征有多个隐向量
- 使用那个，取决于和那个向量进行点乘
- 计算准确度较高，计算速度较慢

## Thinking3：DeepFM相比于FM解决了哪些问题，原理是怎样的


- DeepFM 解决了 FM 一般只考虑 2 阶特征组合的问题，实现了既考虑低阶（1阶 + 2 阶），又能考虑到高阶特征==> DeepFM = FM + DNN 
- 原理：
1. 提取低阶（low order）的特征 ==> 因子分解机FM既可以做一阶特征建模，也可以做二阶特征建模
2. 提取高阶（high order)特征 ==> 用神经网络 DNN 来提取高阶的特征
3. end-to-end 共享特征输入


## Thinking4：Surprise工具中的baseline算法原理是怎样的？BaselineOnly和KNNBaseline有什么区别？

- Baseline 算法的思想主要就是设立基线，并引入用户偏差以及item的偏差
     
            𝑟 ̂_𝑢𝑖 = 𝑏_𝑢𝑖 = 𝜇 + 𝑏_𝑢 + 𝑏_𝑖
            
            
 1. 𝜇 ：所有用户对电影评分的均值
 2. 𝑏_𝑢𝑖: 待求的基线模型中用户 u 给物体打分的预估值
 3. 𝑏_𝑢 ：user 偏差（如果用户比较严苛，打分相对偏低，则 𝑏_𝑢 < 0,反之，𝑏_𝑢 > 0
 4. 𝑏_𝑖 : item偏差，反映商品受欢迎的程度



- KNNBaseline = KNN + Baseline


KNNBaseline考虑到用户打分的偏差，偏差计算时基于baseline



## Thinking5：基于邻域的协同过滤都有哪些算法，请简述原理

### 基于领域的协同过滤算法的种类：
- Slopeone
- KNNBasic
- KNNWithMeans
- KNNWithZScore
- KNNBaseline

### 基于邻域的协同过滤的原理

#### 1.基于用户的协同过滤（User CF)的原理
- 利用行为的相似度计算用户的相似度
- Step1:找到和目标兴趣相似的用户集合（可以利用 Jaccard相似度、余弦相似度、基于流行度改进的相似度）
- Step2:用户 u 对物品的相似度，等价于 K 个邻居对物品 i 的兴趣度
- Step3:为用户生成推荐列表


#### 2.基于物品的协同过滤（ItenmCF )
- 利用行为的相似度计算物品的相似度
- Step1:计算物品之间的相似度
- Step2:用户 u 对物品的兴趣度，等价于物品 i 的 K 个邻居物品，受到用户 u 的兴趣度
- Step3:和用户历史上感兴趣的物品越相似的物品，越有可能在用户的推荐列表中获得比较高的排名


预测用户 u 对物体的兴趣度，去掉用户 U 已经喜欢过的物品，剩下按照从大到小进行排序

## Action1：使用libfm工具对movielens进行评分预测，采用SGD优化算法


- step1:对 rating.csv  转化为 libfm 格式


`perl triple_format_to_libfm.pl -in ratings.dat -target 2 -delete_column 3 -separator "::"
  `


- step2:使用 libfm 进行训练 ，输出结果文件 Movielens_predict_out.txt

`libFM -task r -train ratings.dat.libfm  -test ratings.dat.libfm  -dim ’1,1,8’ -iter 100 -method sgd -learn_rate 0.01 -regular ’0,0,0.01’ -init_stdev 0.1 -out movielens_out.txt`
#### P.S:结果见附件 movielens_out.csv

## Action2：使用基于邻域的协同过滤（KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline中的任意一种）对MovieLens数据集进行协同过滤，采用k折交叉验证(k=3)，输出每次计算的RMSE, MAE


In [23]:
# 目前colab已经集成加载google drive云端硬盘的功能了,为了交作业有输出，我又在本地跑了下
from surprise import KNNWithMeans
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import KFold
import pandas as pd
import time


time1=time.time()
# 数据读取
reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
data = Dataset.load_from_file('F:\\Jupyter_notebook_workdir\\BI\\week3\\MovieLens\\ratings.csv', reader=reader)
train_set = data.build_full_trainset()

# 使用 KNNWithMeans
algo = KNNWithMeans(k=50, sim_options={'user_based': False, 'verbose': 'True'})

# 定义K折交叉验证迭代器，K=3
kf = KFold(n_splits=3)
print(' KNNWithMeans 的RMSE值 和 MAE 值为：')
for trainset, testset in kf.split(data):
    # 训练并预测
    algo_funk.fit(trainset)  
    predictions_funk = algo_funk.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions_funk, verbose=True)
    accuracy.mae(predictions_funk, verbose=True)



uid = str(196)
iid = str(302)
# 输出uid对iid的预测结果
print('使用  KNNWithMeans 预测的结果是：')
pred = algo_funk.predict(uid, iid, r_ui=4, verbose=True)

time2=time.time()
print(time2-time1)

 KNNWithMeans 的RMSE值 和 MAE 值为：
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8549
MAE:  0.6537
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8570
MAE:  0.6555
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8568
MAE:  0.6554
使用  KNNWithMeans 预测的结果是：
user: 196        item: 302        r_ui = 4.00   est = 4.04   {'actual_k': 50, 'was_impossible': False}
357.51154112815857
